<a href="https://colab.research.google.com/github/shantanu-sp/Paraphrase_Detection_on_PAWS/blob/phase1/NLP92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=80dde4f83302350bd06d13998862e735672735bad1f5734aae0acaeb1ea7ff99
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [0]:
!pip install transformers

     |████████████████████████████████| 501kB 15.7MB/s 
     |████████████████████████████████| 870kB 15.4MB/s 
     |████████████████████████████████| 3.7MB 48.2MB/s 
     |████████████████████████████████| 1.0MB 55.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=101a1fcf6e5d3317a5ade4c0900454a4c460c7153060bcad8636565dfec56318
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import torch
from transformers import BertTokenizer,BertForSequenceClassification,AdamW,get_linear_schedule_with_warmup
import tensorflow as tf
import os
import wget
import pandas as pd
import numpy as np

In [0]:
#Get GPU name
gpu=tf.test.gpu_device_name()
print(gpu)

/device:GPU:0


In [0]:
# If GPU available
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device=torch.device("cuda")
    print('There are %d GPU(s) available.' %torch.cuda.device_count())
    print('We will use the GPU:',torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


In [0]:
#check mem usage to not run into OOM error
!nvidia-smi

Mon Mar 23 22:52:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    23W /  75W |    535MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#Download the PAWS Wiki dataset
url='https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz'

if not os.path.exists('./paws_wiki_labeled_final.tar.gz'):
    wget.download(url, './paws_wiki_labeled_final.tar.gz')

In [0]:
if not os.path.exists('./pawsWiki_l_final/'):
    !tar -xvf paws_wiki_labeled_final.tar.gz

final/test.tsv
final/
final/train.tsv
final/dev.tsv


In [0]:
#torch.cuda.empty_cache()

In [0]:
#Get training and validation data
train=pd.read_csv('./final/train.tsv',sep='\t')
print(train.shape)
print(train.head())

val=pd.read_csv('./final/dev.tsv',sep='\t')
test=pd.read_csv('./final/test.tsv',sep='\t')

(49401, 4)
   id  ... label
0   1  ...     0
1   2  ...     1
2   3  ...     0
3   4  ...     1
4   5  ...     1

[5 rows x 4 columns]


In [0]:
#Initialize BertTokenzier
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [0]:
#Data Preprocessing and tensor generation
seed=2

#train=train.sample(n=10000,random_state=seed)
tokenized_train=tokenizer.batch_encode_plus(train.iloc[:,1:3].to_numpy().tolist(),max_length=128,pad_to_max_length=True,return_tensors='pt')
labels_train=torch.tensor(train.label.values[:])

#val=val.sample(n=2000,random_state=seed)
tokenized_val=tokenizer.batch_encode_plus(val.iloc[:,1:3].to_numpy().tolist(),max_length=128,pad_to_max_length=True,return_tensors='pt')
labels_val=torch.tensor(val.label.values[:])

In [0]:
print(tokenized_train['input_ids'].shape)

torch.Size([49401, 128])


In [0]:
print(tokenized_train.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

batch_size=32

train_data=TensorDataset(tokenized_train['input_ids'],tokenized_train['attention_mask'],tokenized_train['token_type_ids'],labels_train)
train_sampler=RandomSampler(train_data)
train_dataloader=DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data=TensorDataset(tokenized_val['input_ids'],tokenized_val['attention_mask'],tokenized_val['token_type_ids'],labels_val)
val_sampler=RandomSampler(val_data)
val_dataloader=DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [0]:
#Bert Model transformer with a single sequence classification layer on top
model=BertForSequenceClassification.from_pretrained('bert-base-cased',num_labels=2,output_attentions=False,output_hidden_states=False)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
#Set learning rate
optimizer=AdamW(model.parameters(),lr=2e-5)

In [0]:
epochs=4

#Training steps is no_of_batches*no_of_epochs
total_steps=len(train_dataloader)*epochs

#Learning rate scheduler
scheduler=get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps = total_steps)

In [0]:
# Function to calculate the accuracy of predictions
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import random

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):
    
    #Put model into training mode
    model.train()

    total_loss=0

    for step, batch in enumerate(train_dataloader):

        #Unpack the training batch
        b_input_ids = batch[0].to(device)
        b_attention_mask=batch[1].to(device)
        b_token_type = batch[2].to(device)
        b_labels = batch[3].to(device)

        #Clear previously calculated gradients before performing a backward pass
        #model.zero_grad()        //Not sure if useful or not

        #Perform a forward pass and get the loss
        outputs=model(b_input_ids,token_type_ids=b_token_type,attention_mask=b_attention_mask,labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()

        #Perform backward pass to calculate gradients
        loss.backward()

        #Clip the norm of the gradients to 1.0.
        #Used to prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        #Update weights
        optimizer.step()

        #Update learning rate
        scheduler.step()

    #Avg loss over training data for an epoch
    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    #### Validation

    #Evaluation mode
    model.eval()

    #Tracking variables 
    val_accuracy=0
    nb_val_steps=0

    #Evaluate data for one epoch
    for batch in val_dataloader:
        
        #Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        #Unpack the inputs from dataloader
        b_input_ids,b_attention_mask, b_token_type, b_labels = batch
        
        #Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():        
            outputs = model(b_input_ids,token_type_ids=b_token_type,attention_mask=b_attention_mask)
        
        #Get the "logits" output by the model. The "logits" are output values prior to applying an activation function like the softmax.
        logits = outputs[0]
        
        #Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        #Calculate the accuracy for this batch of test sentences.
        tmp_val_accuracy = flat_accuracy(logits, label_ids)
        
        #Accumulate the total accuracy.
        val_accuracy += tmp_val_accuracy

        #Track the number of batches
        nb_val_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(val_accuracy/nb_val_steps))



  Average training loss: 0.46
  Accuracy: 0.88

  Average training loss: 0.20
  Accuracy: 0.91

  Average training loss: 0.13
  Accuracy: 0.92

  Average training loss: 0.09
  Accuracy: 0.92
